In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bnlearn as bn

In [ ]:
df = pd.read_excel('exoplanets.xls')

In [ ]:
df.head()

In [ ]:
df.info()

1. Categoriza las variables reales como consideres.

In [ ]:
def categorize_variable(dataframe: pd.DataFrame, variable: str, labels: list) -> None:
    dataframe[variable] = pd.qcut(x=dataframe[variable], q=3, labels=labels)
    return None

In [ ]:
categorize_variable(df, 'radius (m)', ["Small", "Medium", "Big"])
categorize_variable(df, 'mass (kg)', ["Light", "Medium", "Heavy"])
categorize_variable(df, 'gravity', ["Weak", "Medium", "Strong"])
categorize_variable(df, 'avg_temp (C)', ["Cold", "Tempered", "Warm"])
categorize_variable(df, 'rotational_period (h)', ["Slow", "Normal", "Fast"])

2. Mediante la librería bnlearn obtén el grafo de manera automática de la red bayesiana y reflexiona sobre su verosimilitud. Cámbialo manualmente si lo crees conveniente.

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

In [ ]:
DAG = bn.structure_learning.fit(df, methodtype='hc', scoretype='bdeu')
bn.plot(DAG)

In [ ]:
model_edges = [
    ('radius (m)', 'mass (kg)'), # La masa depende de la cantidad de materia en el objeto y su tamaño
    ('mass (kg)', 'gravity'), # La gravedad depende de la cantidad de materia en el objeto
    ('radius (m)', 'gravity'), # La gravedad también depende del tamaño del objeto
    ('mass (kg)', 'atmosphere'), # La masa puede afectar la capacidad del objeto para retener una atmósfera.
    ('gravity', 'atmosphere'), # La gravedad puede afectar la capacidad del objeto para retener una atmósfera.
    ('atmosphere', 'oxygene'), # La presencia de una atmósfera adecuada es un factor importante para la existencia de oxígeno respirable
    ('atmosphere', 'water'), # La presencia de una atmósfera adecuada es un factor importante para la existencia de agua líquida
    ('atmosphere', 'life'),  # La atmósfera es un factor esencial para la vida
    ('oxygene', 'life'), # El oxígeno es un factor esencial para la vida
    ('water', 'life') # El agua es un factor esencial para la vida
]

DAG_life = bn.make_DAG(model_edges)

bn.plot(DAG_life, interactive=False)

3. Busca de manera automática las tablas de probabilidad condicionada de la red.

In [ ]:
model = bn.parameter_learning.fit(DAG, df)

In [ ]:
bn.print_CPD(model)

4. Escoge algunas propiedades de los planetas y realiza la inferencia sobre ellos.

In [ ]:
q1 = bn.inference.fit(model, variables=['life'], evidence={'water': 'YES', 'atmosphere': 'YES'})
print(q1.df)

In [ ]:
q2 = bn.inference.fit(model, variables=['life'], evidence={'water': 'NO', 'atmosphere': 'NO', 'oxygene': 'NO'})
print(q2.df)

In [ ]:
q3 = bn.inference.fit(model, variables=['life'], evidence={'mass (kg)': 'Light', 'radius (m)': 'Small'})
print(q3.df)

In [ ]:
q4 = bn.inference.fit(model, variables=['life'], evidence={'mass (kg)': 'Heavy', 'radius (m)': 'Big'})
print(q4.df)